In [26]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [27]:
!git clone https://github.com/parthmodi444/AI-AGENT.git

Cloning into 'AI-AGENT'...


In [29]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [31]:
!git config --global user.email "modi.parth18@siesgst.ac.in"
!git config --global user.name "parthmodi444"

In [35]:
!git add gen_ai.ipynb

fatal: pathspec 'gen_ai.ipynb' did not match any files


In [32]:
!git add .
!git commit -m "first commit"
!git push

error: 'AI-AGENT/' does not have a commit checked out
fatal: adding files failed
On branch master

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	AI-AGENT/
	drive/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)
fatal: No configured push destination.
Either specify the URL from the command-line or configure a remote repository using

    git remote add <name> <url>

and then push using the remote name

    git push <name>



In [1]:
!pip install openai pandas matplotlib seaborn nltk requests python-dotenv

import os
import json
import time
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from nltk.sentiment import SentimentIntensityAnalyzer
from google.colab import userdata
import nltk

In [2]:
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
NEWS_API_KEY = userdata.get('NEWS_API_KEY')

In [4]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

# Set up paths for RAG documents
RAG_FOLDER = "/content/drive/MyDrive/news_aggregator_rag"
os.makedirs(RAG_FOLDER, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
news_analysis_guidelines = """# News Analysis Methodology

## Principles of News Analysis
1. **Verify sources**: Always check the credibility of sources
2. **Cross-reference information**: Confirm facts across multiple outlets
3. **Identify bias**: Recognize potential biases in reporting
4. **Contextual understanding**: Place news in historical and cultural context
5. **Separate fact from opinion**: Distinguish between factual reporting and commentary

## Key Analytical Frameworks
1. **5W1H Analysis**: Who, What, When, Where, Why, and How
2. **SWOT Analysis**: Strengths, Weaknesses, Opportunities, and Threats
3. **Stakeholder Analysis**: Identify who is affected and how
4. **Trend Analysis**: How does this fit into longer-term patterns?
5. **Comparative Analysis**: How does coverage differ across sources?

## Best Practices for News Summarization
1. Prioritize key information over secondary details
2. Maintain objectivity in presentation
3. Include diverse perspectives when available
4. Acknowledge limitations in available information
5. Update analysis when new information emerges

## Sentiment Analysis in News
1. **Positive sentiment**: Language expressing approval, optimism, progress
2. **Negative sentiment**: Language expressing criticism, pessimism, decline
3. **Neutral sentiment**: Factual reporting without evaluative language
4. Recognize that sentiment can vary by topic, source, and audience

## Types of News Bias
1. **Selection bias**: What stories are covered or ignored
2. **Placement bias**: Where stories appear and how much prominence they receive
3. **Source bias**: Which voices and perspectives are included
4. **Tone bias**: Language choices that subtly influence perception
5. **Visual bias**: How images frame the narrative

## Ethical Considerations
1. Respect privacy and dignity of individuals
2. Consider consequences of amplifying certain stories
3. Avoid sensationalism and fearmongering
4. Provide context for potentially misleading statistics
5. Acknowledge unknowns and uncertainties
"""


In [7]:
with open(f"{RAG_FOLDER}/news_analysis_guidelines.md", "w") as f:
    f.write(news_analysis_guidelines)


In [8]:
news_sources_info = """# Major News Sources Information

## News Source Profiles

### Reuters
**Political Leaning**: Generally Neutral
**Focus Areas**: Global news, financial markets, business
**Notable For**: Factual reporting with minimal bias
**Founding Year**: 1851
**Headquarters**: London, UK

### Associated Press (AP)
**Political Leaning**: Generally Neutral
**Focus Areas**: Breaking news, national and international coverage
**Notable For**: Widely syndicated content, factual reporting
**Founding Year**: 1846
**Headquarters**: New York City, USA

### BBC
**Political Leaning**: Slightly left of center
**Focus Areas**: Global news, UK news, cultural reporting
**Notable For**: Public broadcasting, international perspective
**Founding Year**: 1922
**Headquarters**: London, UK

### Fox News
**Political Leaning**: Right-leaning
**Focus Areas**: US politics, conservative viewpoints
**Notable For**: Opinion-based shows, conservative commentary
**Founding Year**: 1996
**Headquarters**: New York City, USA

### CNN
**Political Leaning**: Left-leaning
**Focus Areas**: US politics, breaking news, international affairs
**Notable For**: 24-hour news cycle, analysis and commentary
**Founding Year**: 1980
**Headquarters**: Atlanta, USA

### The New York Times
**Political Leaning**: Left-leaning
**Focus Areas**: US politics, international affairs, arts
**Notable For**: In-depth reporting, investigative journalism
**Founding Year**: 1851
**Headquarters**: New York City, USA

### The Wall Street Journal
**Political Leaning**: Center-right (news), Right-leaning (editorial)
**Focus Areas**: Business, finance, US politics
**Notable For**: Financial reporting, business analysis
**Founding Year**: 1889
**Headquarters**: New York City, USA

## Understanding Media Bias Ratings

### How Media Bias Is Evaluated
1. **Content Analysis**: Systematic examination of language, sources, and framing
2. **Source Diversity**: Variety of perspectives included in reporting
3. **Factual Accuracy**: Verification of claims against primary sources
4. **Transparency**: Disclosure of methods, sources, and potential conflicts

### Common Bias Classifications
1. **Left**: Favors liberal/progressive perspectives
2. **Lean Left**: Slight preference for liberal perspectives
3. **Center**: Balanced coverage with minimal bias
4. **Lean Right**: Slight preference for conservative perspectives
5. **Right**: Favors conservative perspectives

### Factors That Influence News Coverage
1. **Ownership Structure**: Corporate vs. independent ownership
2. **Revenue Model**: Subscription vs. advertising-based
3. **Target Audience**: Demographics and political leanings
4. **Editorial Policies**: Guidelines for reporting and fact-checking
5. **Geographic Focus**: Local, national, or international emphasis
"""


In [9]:
with open(f"{RAG_FOLDER}/news_sources_info.md", "w") as f:
    f.write(news_sources_info)


In [10]:
def create_vector_store():
    # 1. Create empty vector store
    vector_store = client.vector_stores.create(  # NOT in beta namespace
        name="News Analysis Store",
        chunking_strategy={"type": "auto"}
    )

    # 2. Upload files separately
    file_paths = [
        f"{RAG_FOLDER}/news_analysis_guidelines.md",
        f"{RAG_FOLDER}/news_sources_info.md"
    ]

    # Upload files with assistants purpose
    files = [client.files.create(file=open(path, "rb"), purpose="assistants")
            for path in file_paths]

    # 3. Attach files to vector store
    for file in files:
        client.vector_stores.files.create(  # Direct vector_stores path
            vector_store_id=vector_store.id,
            file_id=file.id
        )

    print(f"Vector store {vector_store.id} ready with {len(files)} files")
    return vector_store.id

# Create vector store
vector_store_id = create_vector_store()

Vector store vs_680cbf0f852481919a7d807023a02e7d ready with 2 files


In [11]:
assistant = client.beta.assistants.create(
    name="News Intelligence Aggregator",
    instructions="""You are a sophisticated News Intelligence Aggregator assistant specialized in gathering, analyzing, and presenting news in a thoughtful and insightful manner.

Your capabilities include:
1. Fetching current news headlines and articles from various sources
2. Analyzing news content for trends, sentiment, and patterns
3. Providing context and background for major news stories
4. Comparing coverage across different news sources
5. Generating visualizations of news data
6. Offering balanced perspectives on complex topics

When analyzing news:
- Always maintain objectivity and avoid political bias
- Consider multiple perspectives on controversial topics
- Verify information across multiple sources when possible
- Distinguish between facts and opinions in your analysis
- Provide context to help users understand the significance of news events
- Be transparent about limitations in available information

Your personality combines the analytical rigor of a seasoned journalist with the contextual understanding of a historian. You are neither sensationalist nor boring - you present information clearly while highlighting truly important developments and patterns.

Use the code interpreter for data analysis, sentiment analysis, and visualization tasks. Use function calls to retrieve fresh news data. Use your knowledge files to apply best practices in news analysis and understand media sources.""",
    model="gpt-3.5-turbo",
    tools=[
        {"type": "file_search"},
        {"type": "code_interpreter"},
        {
            "type": "function",
            "function": {
                "name": "fetch_top_headlines",
                "description": "Fetch top news headlines by country/category",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "country": {"type": "string"},
                        "category": {
                            "type": "string",
                            "enum": ["business", "entertainment", "general", "health", "science", "sports", "technology"]
                        },
                        "query": {"type": "string"},
                        "page_size": {"type": "integer"}
                    },
                    "required": ["country"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "search_news",
                "description": "Search for news articles by keywords and date range",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string"},
                        "from_date": {"type": "string"},
                        "to_date": {"type": "string"},
                        "sort_by": {
                            "type": "string",
                            "enum": ["relevancy", "popularity", "publishedAt"]
                        },
                        "page_size": {"type": "integer"}
                    },
                    "required": ["query"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_news_sources",
                "description": "Get available news sources by country/category",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "country": {"type": "string"},
                        "category": {
                            "type": "string",
                            "enum": ["business", "entertainment", "general", "health", "science", "sports", "technology"]
                        },
                        "language": {"type": "string"}
                    }
                }
            }
        }
    ],
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store_id]
        }
    }
)

In [12]:
def fetch_top_headlines(country='us', category=None, query=None, page_size=10):
    """
    Fetch top headlines from NewsAPI

    Parameters:
        country (str): Country code (e.g., 'us', 'gb', 'in')
        category (str): Category (e.g., 'business', 'technology', 'sports')
        query (str): Search query
        page_size (int): Number of results to return

    Returns:
        dict: JSON response with headlines
    """
    url = "https://newsapi.org/v2/top-headlines"

    params = {
        "apiKey": NEWS_API_KEY,
        "country": country,
        "pageSize": page_size
    }

    if category:
        params["category"] = category

    if query:
        params["q"] = query

    response = requests.get(url, params=params)

    if response.status_code == 200:
        print(f"🌐 Fetched {len(response.json()['articles'])} headlines from {country.upper()}")
        return response.json()
    else:
        return {"error": f"API request failed with status code {response.status_code}: {response.text}"}



In [13]:
def search_news(query, from_date=None, to_date=None, sort_by='publishedAt', page_size=10):
    """
    Search for news articles with NewsAPI

    Parameters:
        query (str): Search query
        from_date (str): Start date in format YYYY-MM-DD
        to_date (str): End date in format YYYY-MM-DD
        sort_by (str): How to sort results ('relevancy', 'popularity', 'publishedAt')
        page_size (int): Number of results to return

    Returns:
        dict: JSON response with news articles
    """
    url = "https://newsapi.org/v2/everything"

    # Default dates if none provided
    if not from_date:
        from_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
    if not to_date:
        to_date = datetime.now().strftime('%Y-%m-%d')


    print(f"📆 Actual dates used: {from_date} to {to_date}")


    params = {
        "apiKey": NEWS_API_KEY,
        "q": query,
        "from": from_date,
        "to": to_date,
        "sortBy": sort_by,
        "pageSize": page_size
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        print(f"🔎 Search results for '{query}': {len(response.json()['articles'])} articles")
        return response.json()
    else:
        return {"error": f"API request failed with status code {response.status_code}: {response.text}"}


In [14]:
def get_news_sources(country=None, category=None, language=None):
    """
    Get available news sources from NewsAPI

    Parameters:
        country (str): Country code (e.g., 'us', 'gb', 'in')
        category (str): Category (e.g., 'business', 'technology', 'sports')
        language (str): Language code (e.g., 'en', 'fr', 'es')

    Returns:
        dict: JSON response with news sources
    """
    url = "https://newsapi.org/v2/sources"

    params = {"apiKey": NEWS_API_KEY}

    if country:
        params["country"] = country

    if category:
        params["category"] = category

    if language:
        params["language"] = language

    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"API request failed with status code {response.status_code}: {response.text}"}

In [15]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "fetch_top_headlines",
            "description": "Fetch top news headlines by country, category, or search query",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "The 2-letter ISO 3166-1 code of the country (e.g., 'us', 'gb', 'in')"
                    },
                    "category": {
                        "type": "string",
                        "enum": ["business", "entertainment", "general", "health", "science", "sports", "technology"],
                        "description": "The category to get headlines for"
                    },
                    "query": {
                        "type": "string",
                        "description": "Keywords or phrases to search for in headlines"
                    },
                    "page_size": {
                        "type": "integer",
                        "description": "Number of results to return (default: 10, max: 100)"
                    }
                },
                "required": ["country"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_news",
            "description": "Search for news articles by keywords, date range, and sorting criteria",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Keywords or phrases to search for in articles"
                    },
                    "from_date": {
                        "type": "string",
                        "description": "Start date in format YYYY-MM-DD (defaults to 7 days ago)"
                    },
                    "to_date": {
                        "type": "string",
                        "description": "End date in format YYYY-MM-DD (defaults to today)"
                    },
                    "sort_by": {
                        "type": "string",
                        "enum": ["relevancy", "popularity", "publishedAt"],
                        "description": "How to sort the articles"
                    },
                    "page_size": {
                        "type": "integer",
                        "description": "Number of results to return (default: 10, max: 100)"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_news_sources",
            "description": "Get available news sources by country, category, or language",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "The 2-letter ISO 3166-1 code of the country (e.g., 'us', 'gb', 'in')"
                    },
                    "category": {
                        "type": "string",
                        "enum": ["business", "entertainment", "general", "health", "science", "sports", "technology"],
                        "description": "The category of news sources"
                    },
                    "language": {
                        "type": "string",
                        "description": "The 2-letter ISO 639-1 code of the language (e.g., 'en', 'fr', 'es')"
                    }
                }
            }
        }
    }
]

# Step 7: Create the assistant
# First, upload the RAG files we created
file_ids = []

In [16]:
thread = client.beta.threads.create()


In [17]:
def process_tool_calls(run, thread_id):


    tool_calls = run.required_action.submit_tool_outputs.tool_calls
    tool_outputs = []

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        print(f"🔧 Calling {function_name} with args: {function_args}")


        output = None
        if function_name == "fetch_top_headlines":
            output = fetch_top_headlines(**function_args)
            if 'articles' in output:
                print(f"🗞️ Retrieved {len(output['articles'])} articles")
        elif function_name == "search_news":
            output = search_news(**function_args)
            if 'articles' in output:
                print(f"🔍 Found {len(output['articles'])} articles for query: '{function_args['query']}'")
        elif function_name == "get_news_sources":
            output = get_news_sources(**function_args)
            if 'sources' in output:
                print(f"📰 Found {len(output['sources'])} sources with params: {function_args}")




        tool_outputs.append({
            "tool_call_id": tool_call.id,
            "output": json.dumps(output)
        })

    # Submit outputs back to the assistant
    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )



In [18]:
def chat_with_assistant(user_message):
    # Add the user's message to the thread
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message
    )

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    # Poll for the run to complete
    while True:
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )

        if run.status == "completed":
            print("Run completed")
            break
        elif run.status == "requires_action":
            print("Run requires action")
            run = process_tool_calls(run, thread.id)
        elif run.status in ["failed", "cancelled", "expired"]:
            print(f"Run ended with status: {run.status}")
            break

        time.sleep(1)

    #rag_keywords = ["media bias", "source profile", "editorial policy"]
    #visualization_extensions = [".png", ".jpg", ".chart"]

    # Get messages (newest first)
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )

    # Return the latest assistant message
    # for message in messages.data:
    #     if message.role == "assistant":
    #         for content_block in message.content:
    #             # Check for RAG usage
    #             if any(keyword in content_block.text.value.lower()
    #                    for keyword in rag_keywords):
    #                 print("\n📚 RAG Context Used: Media bias profiles")

    #             # Check for visualizations
    #             if hasattr(content_block, 'file_id'):
    #                 print(f"\n📈 Visualization Generated: {content_block.file_id}")

    #         return message.content


    for message in messages.data:
      if message.role == "assistant":
        return message.content


In [19]:
def demonstrate_assistant():
    print("=== News Intelligence Aggregator Demo ===\n")

    # Example 1: Get top headlines
    print("Example 1: Getting top technology headlines")
    response = chat_with_assistant("What are the top technology headlines today?")
    for content_block in response:
        if hasattr(content_block, 'text'):
            print(content_block.text.value)
    print("\n" + "-"*50 + "\n")

    # Example 2: Analyze a specific news topic
    print("Example 2: Analyzing coverage of a major news topic")
    response = chat_with_assistant("Can you analyze recent news about climate change and show me the sentiment trends?")
    for content_block in response:
        if hasattr(content_block, 'text'):
            print(content_block.text.value)
    print("\n" + "-"*50 + "\n")

    # Example 3: Compare news sources
    print("Example 3: Comparing coverage across news sources")
    response = chat_with_assistant("Compare how CNN and Fox News are covering the latest political developments.")
    for content_block in response:
        if hasattr(content_block, 'text'):
            print(content_block.text.value)
    print("\n" + "-"*50 + "\n")

# Run the demonstration


In [20]:
def interactive_chat():
    print("Welcome to News Intelligence Aggregator!")
    print("Type 'exit' to end the conversation\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break

        print("\nThinking...")
        response = chat_with_assistant(user_input)

        print("\nNews Intelligence Aggregator:")
        for content_block in response:
            if hasattr(content_block, 'text'):
                print(content_block.text.value)
        print("\n")

In [21]:
def demo_sentiment_analysis():
    """Demonstrate sentiment analysis across different news sources"""
    prompt = """Analyze the sentiment of recent news about artificial intelligence from different sources.
    Show me which sources are more positive or negative, and create a visualization of the sentiment differences."""

    print("Demonstrating sentiment analysis capability...\n")
    print(f"Query: {prompt}\n")

    response = chat_with_assistant(prompt)
    for content_block in response:
        if hasattr(content_block, 'text'):
            print(content_block.text.value)

def demo_topic_tracking():
    """Demonstrate tracking a news topic over time"""
    prompt = """Track how news coverage of Ukraine has evolved over the past week.
    Can you identify any significant shifts in focus or tone? Create a visualization showing the main subtopics."""

    print("Demonstrating topic tracking capability...\n")
    print(f"Query: {prompt}\n")

    response = chat_with_assistant(prompt)
    for content_block in response:
        if hasattr(content_block, 'text'):
            print(content_block.text.value)

def demo_source_comparison():
    """Demonstrate comparing news sources"""
    prompt = """Find news about the economy from both left-leaning and right-leaning sources.
    Compare their coverage and highlight the key differences in how they frame economic issues."""

    print("Demonstrating news source comparison...\n")
    print(f"Query: {prompt}\n")

    response = chat_with_assistant(prompt)
    for content_block in response:
        if hasattr(content_block, 'text'):
            print(content_block.text.value)


In [22]:
print(f"Assistant ID: {assistant.id}")
print(f"Thread ID: {thread.id}")
print("RAG File IDs:", file_ids)
print("\nYour News Intelligence Aggregator is ready to use!")
print("To start using it, run the interactive_chat() function or try one of the demo functions.")

Assistant ID: asst_vNh5eH9fuqTP2QpIOlYxer2u
Thread ID: thread_YGN19pfz4qPtou2nJr9aEwxx
RAG File IDs: []

Your News Intelligence Aggregator is ready to use!
To start using it, run the interactive_chat() function or try one of the demo functions.


In [23]:
def log_visualization(code):
    """Helper to log code interpreter activities"""
    visualization_triggers = [
        "plt.show()",
        "sns.heatmap",
        "ax.plot(",
        "fig.savefig"
    ]

    if any(trigger in code for trigger in visualization_triggers):
        print(f"\n📊 Visualization Code:\n{code}")

In [25]:
interactive_chat()

Welcome to News Intelligence Aggregator!
Type 'exit' to end the conversation



KeyboardInterrupt: Interrupted by user